In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.notebook import tqdm

# 1. Path

In [3]:
# klue/roberta-large
# path = {
#     "유형": ["results/유형/2022-12-20_170839/checkpoint-1854","klue/roberta-large"],
#     "극성": ["results/극성/2022-12-20_184737/checkpoint-1236","klue/roberta-large"],
#     "시제": ["results/시제/2022-12-20_202634/checkpoint-2060","klue/roberta-large"],
#     "확실성": ["results/확실성/2022-12-20_220529/checkpoint-618","klue/roberta-large"],
# }


# # kykim/electra-kor-base
# path = {
#     "유형": ["results/유형/2022-12-22_220850/checkpoint-3255","kykim/electra-kor-base"],
#     "극성": ["results/극성/2022-12-22_224637/checkpoint-2325","kykim/electra-kor-base"],
#     "시제": ["results/시제/2022-12-22_232428/checkpoint-1395","kykim/electra-kor-base"],
#     "확실성": ["results/확실성/2022-12-21_073113/checkpoint-1239","kykim/electra-kor-base"],
# }

# klue/roberta-base
path = {
    "유형": ["results/유형/2022-12-23_012330/checkpoint-1398","klue/roberta-base"],
    "극성": ["results/극성/2022-12-23_020957/checkpoint-1864","klue/roberta-base"],
    "시제": ["results/시제/2022-12-23_025624/checkpoint-1631","klue/roberta-base"],
    "확실성": ["results/확실성/2022-12-23_034237/checkpoint-1165","klue/roberta-base"],
}

# # beomi/KcELECTRA-base-v2022
# path = {
#     "유형": ["results/유형/2022-12-23_012330/checkpoint-1398","beomi/KcELECTRA-base-v2022"],
#     "극성": ["results/극성/2022-12-23_020957/checkpoint-1864","beomi/KcELECTRA-base-v2022"],
#     "시제": ["results/시제/2022-12-23_025624/checkpoint-1395","beomi/KcELECTRA-base-v2022"],
#     "확실성": ["results/확실성/2022-12-23_034237/checkpoint-1165","beomi/KcELECTRA-base-v2022"],
# }

# 2. Data

In [4]:
!cp data/test.csv data/result.csv

In [5]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm
from datasets import load_dataset

tds = load_dataset(
    "csv",
    data_files={"predict": "data/result.csv"}
)

Using custom data configuration default-c03bc7c9068be58e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating predict split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csv/default-c03bc7c9068be58e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
def inference(model, tokenizer, category):
    pipe = pipeline("text-classification", model=model, tokenizer = tokenizer, device=0)
    result = []
    for out in tqdm(pipe(KeyDataset(tds["predict"], "문장"), batch_size=64)):
        result.append(out["label"])
    return result

In [7]:
for category, (pretrained_model_name_or_path, plm) in path.items():
    tokenizer = AutoTokenizer.from_pretrained(plm)
    model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path)
    result = inference(model, tokenizer, category)
    tds["predict"] = tds["predict"].add_column(name=category, column=result)

7090it [00:16, 438.77it/s]                      
7090it [00:15, 454.94it/s]                       
7090it [00:15, 453.39it/s]                       
7090it [00:15, 448.10it/s]                       


In [8]:
tds["predict"] = tds["predict"].map(lambda batch: {"label": "-".join([batch["유형"], batch["극성"], batch["시제"], batch["확실성"]])}, \
                                  remove_columns=list(path.keys()) + ["문장"])

  0%|          | 0/7090 [00:00<?, ?ex/s]

In [9]:
tds["predict"]

Dataset({
    features: ['ID', 'label'],
    num_rows: 7090
})

In [10]:
tds["predict"].to_csv("klue-base-submission.csv", encoding="UTF-8", index=False)

Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

292574